# Primer: TOML implementation
This notebook reconstructs the `Translator` showcased in the [Translation primer](../../../translation-primer.rst) using the a [TOML configuration](config.toml).

In [1]:
import sys
import rics
import id_translation

# Print relevant versions
print(f"{rics.__version__=}")
print(f"{id_translation.__version__=}")
print(f"{sys.version=}")
rics.configure_stuff(id_translation_level="DEBUG")
!git log --pretty=oneline --abbrev-commit -1

rics.__version__='3.0.0'
id_translation.__version__='0.3.1.dev1'
sys.version='3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]'
👻 Configured some stuff just the way I like it!
d2093a4 (HEAD, origin/main, origin/HEAD, main) Update help link in TOML files


## Translatable data

In [2]:
from pandas import read_csv

bite_report = read_csv("biting-victims-2019-05-11.csv")
bite_report

,human_id,bitten_by
0,1904,1
1,1991,0
2,1991,2
3,1999,0


## Define heuristic score function

In [3]:
from id_translation.fetching import PandasFetcher
from rics.mapping import HeuristicScore, Mapper


def smurf_column_heuristic(value, candidates, context):
    """Heuristic for matching columns that use the "smurf" convention.

    The value is the desired placeholder, the context is the name of the source
    for which placeholder mapping is being performed.
    """
    return (
        f"{context[:-1]}_{value}" if context[-1] == "s" else f"{context}_{value}",
        candidates,
    )

## Moment of truth

In [4]:
from id_translation import Translator

translated_bite_report = Translator.from_config("config.toml").translate(bite_report)
translated_bite_report

2023-03-25T11:21:35.313 [id_translation.fetching.config-toml.pandas.discovery:DEBUG] Sources initialized: ['humans', 'animals']
2023-03-25T11:21:35.316 [id_translation.Translator:INFO] Begin translation of 'DataFrame' using sources=['humans', 'animals']. Names to translate: Will be derived based on 'DataFrame'.
2023-03-25T11:21:35.316 [id_translation.Translator:DEBUG] Begin name-to-source mapping of names=['human_id', 'bitten_by'] in DataFrame against sources=['humans', 'animals'].
2023-03-25T11:21:35.317 [id_translation.mapping.name-to-source:DEBUG] Begin computing match scores for ['human_id', 'bitten_by']x['humans', 'animals'] using HeuristicScore([like_database_table()] -> equality).
2023-03-25T11:21:35.317 [id_translation.mapping.name-to-source:DEBUG] Applied 1 overrides, and found 1 matches={'bitten_by': 'animals'} in the given values=['human_id', 'bitten_by'].
2023-03-25T11:21:35.319 [id_translation.Translator:DEBUG] Finished name-to-source mapping of names=['human_id', 'bitten_

,human_id,bitten_by
0,Mr. Fred (id=1904),Morris (id=1) the dog
1,Mr. Richard (id=1991),Tarzan (id=0) the cat
2,Mr. Richard (id=1991),Simba (id=2) the lion
3,Dr. Sofia (id=1999),Tarzan (id=0) the cat


In [5]:
assert translated_bite_report.equals(
    read_csv("biting-victims-2019-05-11-translated.csv")
)

## Print the config
Click [here](config.toml) to download.

In [6]:
!pygmentize config.toml

################################################################################
# For help, see https://id-translation.readthedocs.io                          #
################################################################################
[translator]
fmt = "[{title}. ]{name} (id={id})[ the {species}]"

# ------------------------------------------------------------------------------
# Name-to-source mapping configuration. Binds names to source, eg 'cute_animals'
# -> 'my_database.animals'. Overrides take precedence over scoring logic.
[translator.mapping]
score_function="equality"
[[translator.mapping.score_function_heuristics]]
function = "like_database_table"

[translator.mapping.overrides]
bitten_by = "animals"

################################################################################
# Fetching configuration.
################################################################################
[fetching.PandasFetcher]
read_function = "read_csv"
read_path_format= "./sources/{}